In [34]:
import os, math
from IPython.display import Image
import numpy as np
seed = np.random.seed
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [35]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [36]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [37]:
normalize = 255
x_train = x_train/normalize
x_test = x_test/normalize

In [38]:
enc = OneHotEncoder()
print(y_train)

[5 0 4 ... 5 6 8]


In [39]:
yy_train_y = y_train.reshape(-1,1)
print(yy_train_y)

[[5]
 [0]
 [4]
 ...
 [5]
 [6]
 [8]]


In [40]:
enc.fit(yy_train_y)
yy_hot_full_train = enc.transform(yy_train_y).toarray()

In [41]:
yy_test_y = y_test.reshape(-1,1)
enc.fit(yy_test_y)
yy_hot_full_test = enc.transform(yy_test_y).toarray()

In [42]:
mlp = MLPClassifier(hidden_layer_sizes = (50,), max_iter = 1000, alpha = 1e-4,
                   solver = 'adam', verbose = 10, random_state = 1, learning_rate_init = 0.1, tol = 0.1)

mlp.out_activation_ = 'softmax'

In [43]:
mlp.fit(x_train, yy_hot_full_train)

Iteration 1, loss = 1.19203757
Iteration 2, loss = 0.78728511
Iteration 3, loss = 0.75479509
Iteration 4, loss = 0.75738864
Iteration 5, loss = 0.75038194
Iteration 6, loss = 0.75118480
Iteration 7, loss = 0.74215261
Iteration 8, loss = 0.76120685
Iteration 9, loss = 0.76680486
Iteration 10, loss = 0.75487867
Iteration 11, loss = 0.76569301
Iteration 12, loss = 0.75410941
Iteration 13, loss = 0.75126379
Training loss did not improve more than tol=0.100000 for 10 consecutive epochs. Stopping.


MLPClassifier(hidden_layer_sizes=(50,), learning_rate_init=0.1, max_iter=1000,
              random_state=1, tol=0.1, verbose=10)

In [44]:
print("Training set score: %f" % mlp.score(x_train, yy_hot_full_train))
print("Test set score: %f" % mlp.score(x_test, yy_hot_full_test))

Training set score: 0.831783
Test set score: 0.822800


In [45]:
parameters = {'learning_rate_init': (0.1, 0.01),
             'hidden_layer_sizes': (20, 100),
             'solver': ('lbfgs', 'sgd', 'adam')}

In [46]:
clf = GridSearchCV(mlp, parameters)
clf.fit(x_train, yy_hot_full_train)

ss = 0.39998596
Iteration 9, loss = 0.39265065
Iteration 10, loss = 0.38489124
Iteration 11, loss = 0.37988924
Iteration 12, loss = 0.37736222
Iteration 13, loss = 0.37119549
Training loss did not improve more than tol=0.100000 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.88195112
Iteration 2, loss = 0.54886724
Iteration 3, loss = 0.50339365
Iteration 4, loss = 0.47057993
Iteration 5, loss = 0.45149661
Iteration 6, loss = 0.43634430
Iteration 7, loss = 0.42572195
Iteration 8, loss = 0.41544984
Iteration 9, loss = 0.40885308
Iteration 10, loss = 0.39915186
Iteration 11, loss = 0.39572276
Iteration 12, loss = 0.39139258
Iteration 13, loss = 0.38512772
Training loss did not improve more than tol=0.100000 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.88083391
Iteration 2, loss = 0.54447423
Iteration 3, loss = 0.49975530
Iteration 4, loss = 0.47378904
Iteration 5, loss = 0.45250235
Iteration 6, loss = 0.43734229
Iteration 7, loss = 0.42101456
Iteration 8, loss = 0

GridSearchCV(estimator=MLPClassifier(hidden_layer_sizes=(50,),
                                     learning_rate_init=0.1, max_iter=1000,
                                     random_state=1, tol=0.1, verbose=10),
             param_grid={'hidden_layer_sizes': (20, 100),
                         'learning_rate_init': (0.1, 0.01),
                         'solver': ('lbfgs', 'sgd', 'adam')})

In [47]:
print("Training set score: %f" % clf.score(x_train, yy_hot_full_train))
print("Test set score: %f" % clf.score(x_test, yy_hot_full_test))

Training set score: 0.985267
Test set score: 0.957400


In [48]:
print(clf.best_params_)

{'hidden_layer_sizes': 100, 'learning_rate_init': 0.1, 'solver': 'sgd'}


In [49]:
X = np.vstack((x_train, x_test))
y = np.hstack((y_train, y_test))

In [50]:
y_predict = clf.predict(X)

In [51]:
confusion_matrix(y, y_predict.argmax(axis = 1))

array([[6889,    0,    2,    1,    1,    0,    3,    2,    2,    3],
       [  24, 7840,    3,    0,    0,    0,    3,    5,    2,    0],
       [  52,   15, 6910,    1,    0,    1,    1,    8,    2,    0],
       [  90,    6,   28, 7005,    0,    4,    0,    3,    3,    2],
       [  41,   10,    7,    1, 6746,    0,    2,    2,    1,   14],
       [  97,    3,    2,   21,    2, 6176,    7,    0,    3,    2],
       [  60,   14,    2,    0,    7,    5, 6788,    0,    0,    0],
       [  46,   19,   22,   11,    5,    0,    0, 7187,    1,    2],
       [ 169,   10,    9,   14,    3,    5,    6,    5, 6603,    1],
       [ 117,    6,    2,   17,   17,    3,    0,   24,    9, 6763]],
      dtype=int64)

In [52]:
sns.heatmap(cm, center=True)
plt.show()

NameError: name 'cm' is not defined